In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.3 MB/s eta 0:00:0000:01


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import Sastrawi
from wordcloud import WordCloud

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-product-at-e-commerce-tokopedia/data_real.csv


In [3]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "data_real.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "faceit24/sentiment-analysis-product-at-e-commerce-tokopedia",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

First 5 records:    Unnamed: 0.1  Unnamed: 0  \
0             0          59   
1             1          70   
2             2          95   
3             3         117   
4             4         153   

                                              Ulasan  Rating     Kategori  \
0                  Pesanan gx sesuai dgn yg d gambar       2  pertukangan   
1  Pengiriman dan respon super cepat. Tpi barang ...       2  pertukangan   
2             barang yg dikirim tidak sesuai pesanan       1  pertukangan   
3  Php, bilang isi ada diseskripsi pas dipesen be...       1  pertukangan   
4    barang jelek copot&#34; terus tmpat taro isinya       2  pertukangan   

                                         Nama Produk  Id Produk Terjual  \
0                 Isi Staples Tembak 8 mm best guard  133507638     545   
1                 Isi Staples Tembak 8 mm best guard  133507638     545   
2                 Isi Staples Tembak 8 mm best guard  133507638     545   
3  steples tembak / staples temba

/tmp/ipykernel_31/1714347155.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


In [4]:
df_copy = df.copy()

## B. Labeling

In [5]:
# Pengkondisian terhadap rating data online shop

label = []
for index, row in df_copy.iterrows():
    if row['Rating'] == 1 or row["Rating"] == 2:
        label.append(0)
    else :
        label.append(1)

df_copy["label"] = label

In [6]:
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


## C. Preprocessing Teks

In [7]:
#def casefolding(Review):
 #   Review = Review.lower()
    #  return Review

#df_copy['Ulasan'] = df_copy['Ulasan'].apply(casefolding)

# Mengubah text menjadi lower

df_copy['Ulasan'] = df_copy['Ulasan'].str.lower()
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,pengiriman dan respon super cepat. tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


In [8]:
norm = {" dgn ":" dengan ", " gue ":" saya ", "bgmn ":" bagaimana ", " tdk":" tidak ", " blum":" belum ", "mantaaaaaaaaaaaappp":" bagus ", "trims":" terima kasih ", "brg":" barang ", "gx":" tidak ", " recommended":" rekomen ", "recomend":"rekomen", "good":" bagus ", " tpi ":" tapi ", " yg ":" yang "}

def normalisasi(str_text):
    for i in norm:
        str_text = str_text.replace(i, norm[i])
    return str_text

df_copy['Ulasan'] = df_copy['Ulasan'].apply(lambda x: normalisasi(x))

df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,pesanan tidak sesuai dengan yang d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,pengiriman dan respon super cepat. tapi barang...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yang dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


## C2. StopWords

In [9]:
# Menghapus kata yang memiliki informasi yang rendah. ex: yang, dan, di, dari, dll

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

more_stop_words = []

stop_words = StopWordRemoverFactory().get_stop_words()
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
    str_text = stop_words_remover_new.remove(str_text)
    return str_text

df_copy['Ulasan'] = df_copy['Ulasan'].apply(lambda x: stopword(x))
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,pesanan sesuai yang d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,pengiriman respon super cepat. barang sesuai g...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang dikirim sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"php, bilang isi diseskripsi pas dipesen berbed...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


## C3. Tokenize

In [10]:
# Memisahkan kalimat, paragraf atau dokumen menjadi kata-kata / bagian tertentu, ex: produk ini bagus: ['produk', 'ini', 'bagus']

tokenized = df_copy['Ulasan'].apply(lambda x:x.split())
tokenized

0                      [pesanan, sesuai, yang, d, gambar]
1       [pengiriman, respon, super, cepat., barang, se...
2                      [barang, dikirim, sesuai, pesanan]
3       [php,, bilang, isi, diseskripsi, pas, dipesen,...
4       [barang, jelek, copot&#34;, terus, tmpat, taro...
                              ...                        
1920           [bagus, memuaskan., packing, bagus, cepat]
1921               [mantap, pengiriman, cepat, rekomened]
1922                        [pengiriman, cepat., lumayan]
1923              [belom, dicoba, semoga, oke, barangnya]
1924    [barang, cukup, bagus,, respon, bagus., gak, m...
Name: Ulasan, Length: 1925, dtype: object

## D. Stemming

In [11]:
# Digunakan untuk memperkecil jumlah indeks yang berbeda dari satu data sehingga sebuah kata yang memiliki suffix maupun prefix akan kembali ke bentuk dasarnya, ex: pesanan -> pesan, membeli -> beli

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(Ulasan):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in Ulasan:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

# tokenized = tokenized.apply(stemming)

tokenized.to_csv('databersih2.csv', index=False)
data_clean = pd.read_csv('databersih2.csv', encoding='latin1')
data_clean.head()

pesan sesuai yang d gambar
kirim respon super cepat barang sesuai gambar tidak kuat tembus jok motor
barang kirim sesuai pesan
php bilang isi skripsi pas sen beda mubajir rp50 000 nyata isi ga cocok
barang jelek copot 34 terus tmpat taro isi
beli staples gak jual isi sekali pake ckckck
baik kalau mau ngirim barang periksa dulu gan kondisi barang layak apa gak gitu loh isi karat semua
produk sesuai order prohex kirim
barang sdh terima
bisa buat masang jok gak pake
barang datang cacat gak guna harus sblum kirim di coba dulu
kecewa baru dipakey isiya kluar 2 3 buah papan nempel
barang kurang bagus kirim pakai pos lama sekali
hasil staples rapat bidang mungkin produk cacat hadeeuh
barang standart n biasa aja kurang bagus 
barang rusak kecewa berat
beli 5 bs pakai semua
straples nya gk bs dipake udh coba berkali2 bwt pasang jaring nyamuk straples tembak bah triplek aja gk nembus udh harus diketok2 palu lgi 
mungkin harga juga jadi susah dipake hehe staples gak nancep sampe oenuh
gagal
ggdjk

,Ulasan
0,pesan sesuai yang d gambar
1,kirim respon super cepat barang sesuai gambar ...
2,barang kirim sesuai pesan
3,php bilang isi skripsi pas sen beda mubajir rp...
4,barang jelek copot 34 terus tmpat taro isi


## Menggabungkan kedua atribut

In [18]:
at1 = pd.read_csv('/kaggle/working/databersih2.csv')
at2 = df
# at2.head()
att2 = at2['label']

result = pd.concat([at1, att2], axis=1)
result.head()

,Ulasan,label
0,pesan sesuai yang d gambar,0
1,kirim respon super cepat barang sesuai gambar ...,0
2,barang kirim sesuai pesan,0
3,php bilang isi skripsi pas sen beda mubajir rp...,0
4,barang jelek copot 34 terus tmpat taro isi,0


## E. Menghitung Kata Dengan TF-IDF

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

Ulasan = result['Ulasan']

Ulasan.isnull().sum()

6

In [22]:
Ulasan = Ulasan.fillna('tidak ada komentar')

cv = CountVectorizer()
term_fit = cv.fit(Ulasan)

print(len(term_fit.vocabulary_))

2856


In [23]:
term_fit.vocabulary_

{'pesan': 1981,
 'sesuai': 2322,
 'yang': 2839,
 'gambar': 914,
 'kirim': 1306,
 'respon': 2161,
 'super': 2497,
 'cepat': 566,
 'barang': 301,
 'tidak': 2635,
 'kuat': 1377,
 'tembus': 2576,
 'jok': 1168,
 'motor': 1673,
 'php': 1987,
 'bilang': 409,
 'isi': 1106,
 'skripsi': 2381,
 'pas': 1921,
 'sen': 2298,
 'beda': 349,
 'mubajir': 1685,
 'rp50': 2191,
 '000': 1,
 'nyata': 1819,
 'ga': 900,
 'cocok': 596,
 'jelek': 1148,
 'copot': 615,
 '34': 69,
 'terus': 2600,
 'tmpat': 2662,
 'taro': 2550,
 'beli': 360,
 'staples': 2454,
 'gak': 911,
 'jual': 1175,
 'sekali': 2274,
 'pake': 1899,
 'ckckck': 582,
 'baik': 276,
 'kalau': 1205,
 'mau': 1586,
 'ngirim': 1765,
 'periksa': 1975,
 'dulu': 818,
 'gan': 919,
 'kondisi': 1346,
 'layak': 1427,
 'apa': 217,
 'gitu': 957,
 'loh': 1479,
 'karat': 1225,
 'semua': 2297,
 'produk': 2049,
 'order': 1863,
 'prohex': 2053,
 'sdh': 2254,
 'terima': 2592,
 'bisa': 413,
 'buat': 485,
 'masang': 1576,
 'datang': 668,
 'cacat': 521,
 'guna': 987,
 'haru

In [25]:
# Kolom pertama ini berarti jumlah dokumen
# Kolom kedua berarti letak kata
# Kolom ketiga berarti hasil dari tf

term_frequency_all = term_fit.transform(Ulasan)
print(term_frequency_all)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 16167 stored elements and shape (1925, 2856)>
  Coords	Values
  (0, 914)	1
  (0, 1981)	1
  (0, 2322)	1
  (0, 2839)	1
  (1, 301)	1
  (1, 566)	1
  (1, 914)	1
  (1, 1168)	1
  (1, 1306)	1
  (1, 1377)	1
  (1, 1673)	1
  (1, 2161)	1
  (1, 2322)	1
  (1, 2497)	1
  (1, 2576)	1
  (1, 2635)	1
  (2, 301)	1
  (2, 1306)	1
  (2, 1981)	1
  (2, 2322)	1
  (3, 1)	1
  (3, 349)	1
  (3, 409)	1
  (3, 596)	1
  (3, 900)	1
  :	:
  (1924, 635)	1
  (1924, 735)	1
  (1924, 898)	1
  (1924, 911)	2
  (1924, 919)	1
  (1924, 959)	1
  (1924, 987)	1
  (1924, 1024)	1
  (1924, 1457)	1
  (1924, 1567)	1
  (1924, 1709)	1
  (1924, 1798)	1
  (1924, 1903)	1
  (1924, 1924)	1
  (1924, 1994)	1
  (1924, 2161)	1
  (1924, 2230)	1
  (1924, 2254)	1
  (1924, 2285)	1
  (1924, 2371)	1
  (1924, 2513)	3
  (1924, 2575)	1
  (1924, 2652)	1
  (1924, 2687)	1
  (1924, 2773)	1


In [26]:
ulasan_tf = Ulasan[1]
print(ulasan_tf)

kirim respon super cepat barang sesuai gambar tidak kuat tembus jok motor


In [28]:
term_frequency = term_fit.transform([ulasan_tf])
print(term_frequency)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12 stored elements and shape (1, 2856)>
  Coords	Values
  (0, 301)	1
  (0, 566)	1
  (0, 914)	1
  (0, 1168)	1
  (0, 1306)	1
  (0, 1377)	1
  (0, 1673)	1
  (0, 2161)	1
  (0, 2322)	1
  (0, 2497)	1
  (0, 2576)	1
  (0, 2635)	1


In [29]:
dokumen = term_fit.transform(Ulasan)
tfidf_transformer = TfidfTransformer().fit(dokumen)
print(tfidf_transformer.idf_)

tfidf = tfidf_transformer.transform(term_frequency)
print(tfidf)

[7.87005341 7.87005341 7.87005341 ... 7.87005341 7.87005341 7.87005341]
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12 stored elements and shape (1, 2856)>
  Coords	Values
  (0, 2635)	0.2563292160896024
  (0, 2576)	0.41620437432092156
  (0, 2497)	0.3435539298709763
  (0, 2322)	0.13966501029508957
  (0, 2161)	0.21043773739326385
  (0, 1673)	0.41620437432092156
  (0, 1377)	0.3289796605395475
  (0, 1306)	0.15721666646860907
  (0, 1168)	0.41620437432092156
  (0, 914)	0.23895960222089352
  (0, 566)	0.17639708772567406
  (0, 301)	0.10786616198641989
